In [1]:
# Import Libraries

import pandas as pd
import numpy as np

# Halo

In [2]:
# Ambil data csv

property_interaction = pd.read_csv('data_percobaan_jurnal_nida.csv',index_col=0)
property_interaction

,property_id,ratings
user_id,,
u1,b2,5
u1,b3,4
u1,b4,3
u2,b3,3
u2,b4,2
u2,b5,4
u2,b6,1
u3,b2,3
u3,b6,3


In [3]:
# Mengubah csv menjadi dataframe

property_interaction_df = pd.DataFrame(property_interaction)
property_interaction_df

,property_id,ratings
user_id,,
u1,b2,5
u1,b3,4
u1,b4,3
u2,b3,3
u2,b4,2
u2,b5,4
u2,b6,1
u3,b2,3
u3,b6,3


In [4]:
#Membuat matriks interaksi

interactions_matrix = pd.pivot_table(property_interaction_df, values='ratings', index='user_id', columns='property_id', aggfunc=np.max)
interactions_matrix

property_id,b1,b2,b3,b4,b5,b6
user_id,,,,,,
u1,NaN,5.0,4.0,3.0,NaN,NaN
u2,NaN,NaN,3.0,2.0,4.0,1.0
u3,NaN,3.0,NaN,NaN,NaN,3.0
u4,4.0,NaN,NaN,1.0,NaN,NaN
u5,NaN,2.0,2.0,4.0,NaN,5.0
u6,NaN,5.0,NaN,4.0,NaN,NaN


In [5]:
# mengisi missing data dengan 0

interactions_matrix = interactions_matrix.fillna(0)
interactions_matrix

property_id,b1,b2,b3,b4,b5,b6
user_id,,,,,,
u1,0.0,5.0,4.0,3.0,0.0,0.0
u2,0.0,0.0,3.0,2.0,4.0,1.0
u3,0.0,3.0,0.0,0.0,0.0,3.0
u4,4.0,0.0,0.0,1.0,0.0,0.0
u5,0.0,2.0,2.0,4.0,0.0,5.0
u6,0.0,5.0,0.0,4.0,0.0,0.0


In [36]:
# import cosine_similarity from sklearn
from sklearn.metrics.pairwise import cosine_similarity

def similar_users(user_id, interactions_matrix):
    
    # compute similarity of each user to the provided user
    similarity = []
    for user in range(0, interactions_matrix.shape[0]):
        sim = cosine_similarity([interactions_matrix.loc[user_id]], [interactions_matrix.loc[interactions_matrix.index[user]]])
        similarity.append((interactions_matrix.index[user], sim))
        print(sim)
        print(interactions_matrix.loc[user_id])
    
    # sort by similarity
    similarity.sort(key=lambda x: x[1], reverse=True)
    
    # create list of just the user ids
    most_similar_users = [tup[0] for tup in similarity]
    
    # create list of similarity score
    similarity_score = [tup[1] for tup in similarity]
    
    # remove the user's own id
    most_similar_users.remove(user_id)
    
    # remove the user's own similarity score
    similarity_score.remove(similarity_score[0])
       
    return most_similar_users, similarity_score
similar_users('u6', interactions_matrix)[1]

[[0.81719329]]
property_id
b1    0.0
b2    5.0
b3    0.0
b4    4.0
b5    0.0
b6    0.0
Name: u6, dtype: float64
[[0.22810638]]
property_id
b1    0.0
b2    5.0
b3    0.0
b4    4.0
b5    0.0
b6    0.0
Name: u6, dtype: float64
[[0.55215763]]
property_id
b1    0.0
b2    5.0
b3    0.0
b4    4.0
b5    0.0
b6    0.0
Name: u6, dtype: float64
[[0.1515108]]
property_id
b1    0.0
b2    5.0
b3    0.0
b4    4.0
b5    0.0
b6    0.0
Name: u6, dtype: float64
[[0.58007397]]
property_id
b1    0.0
b2    5.0
b3    0.0
b4    4.0
b5    0.0
b6    0.0
Name: u6, dtype: float64
[[1.]]
property_id
b1    0.0
b2    5.0
b3    0.0
b4    4.0
b5    0.0
b6    0.0
Name: u6, dtype: float64


[array([[0.81719329]]),
 array([[0.58007397]]),
 array([[0.55215763]]),
 array([[0.22810638]]),
 array([[0.1515108]])]

In [28]:
halo = cosine_similarity([interactions_matrix.loc[u1]], [interactions_matrix.loc[interactions_matrix.index[0]]]

SyntaxError: unexpected EOF while parsing (<ipython-input-28-1480505415bb>, line 1)

In [21]:
# check the implemented function above

similar_users('u6', interactions_matrix)[1]

[array([[0.81719329]]),
 array([[0.58007397]]),
 array([[0.55215763]]),
 array([[0.22810638]]),
 array([[0.1515108]])]

In [ ]:
def recommendations(user_id, num_of_property, user_item_interactions):
    # find the most similar users to the user_id for which we want to recommend movies
    most_similar_users = similar_users(user_id, user_item_interactions)[0]
    
    # find out those movies which this user has already interacted with
    property_ids = set(list(interactions_matrix.columns[np.where(interactions_matrix.loc[user_id] > 0)]))
    
    # create an empty list to store the recommended movies
    recommendations = []
    
    # copy those movies which are already interacted by user_id
    already_interacted = property_ids.copy()
    
    # loop through each similar user from the list of most_similar_users
    for similar_user in most_similar_users:
        
        # implement the below code till the length of recommended movies does not become equal to num_of_movies
        if len(recommendations) < num_of_property:
            
            # store all the movies interacted by each similar user to user_id
            similar_user_property_ids = set(list(interactions_matrix.columns[np.where(interactions_matrix.loc[similar_user] > 0)]))
            
            # add those movies in the recommended list which are present in similar_user_movie_ids but not present in already_interacted
            recommendations.extend(list(similar_user_property_ids.difference(already_interacted)))
            
            # now add all those movies into already_interacted which we already added in recommendations
            already_interacted = already_interacted.union(similar_user_property_ids)
            
        else:
            break
    
    return recommendations[:num_of_property]

In [ ]:
# check the implemented function above

recommendations('5bfaed099a291d00012035eb', 5, interactions_matrix)

# ERROR

In [ ]:
interactions_matrix.loc['5bfaed099a291d00012035eb']

In [ ]:
interactions_matrix.index[1].loc

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def similar_users(user_id, interactions_matrix):

    similarity = []

#sim = cosine_similarity([interactions_matrix.loc['5bfaed099a291d00012035eb']], [interactions_matrix.loc['5bf0b0622169690001e8c106']])
    
    for user in range(0, interactions_matrix.shape[0]):
        sim = cosine_similarity([interactions_matrix.loc[user_id]], [interactions_matrix.loc[interactions_matrix.index[user]]])
        similarity.append((interactions_matrix.index[user], sim))
# sort by similarity
    similarity.sort(key=lambda x: x[1], reverse=True)

# create list of just the user ids
    most_similar_users = [tup[0] for tup in similarity]

# create list of similarity score
    similarity_score = [tup[1] for tup in similarity]

# remove the user's own id
    most_similar_users.remove('5bfaed099a291d00012035eb')
    
# remove the user's own similarity score
    similarity_score.remove(similarity_score[0])
    
    #most_similar_users
    
    return most_similar_users, similarity_score
#sim

In [ ]:
similar_users('5bfaed099a291d00012035eb', interactions_matrix)[1][:20]

# Error

In [ ]:
interactions_matrix.loc['5bfaed099a291d00012035eb']

In [ ]:
interactions_matrix.loc[interactions_matrix.index[1]]
#interactions_matrix.index[1]

In [ ]:
# import cosine_similarity from sklearn
from sklearn.metrics.pairwise import cosine_similarity

def similar_users(interactions_matrix):
    
    # compute similarity of each user to the provided user
    similarity = []
    for user in range(1, interactions_matrix.shape[0]+1):
        sim = cosine_similarity([interactions_matrix.loc['5bf0e7db2169690001e8c10a']], [interactions_matrix.index[user]])
        similarity.append((user, sim))
    
    # sort by similarity
    similarity.sort(key=lambda x: x[1], reverse=True)
    
    # create list of just the user ids
    most_similar_users = [tup[0] for tup in similarity]
    
    # create list of similarity score
    similarity_score = [tup[1] for tup in similarity]
    
    # remove the user's own id
    most_similar_users.remove(user_id)
    
    # remove the user's own similarity score
    similarity_score.remove(similarity_score[0])
       
    return most_similar_users, similarity_score

In [ ]:
similar_users(interactions_matrix)[0][:5]

In [ ]:
#for i in range(0, property_interaction.shape[0]):
#    for y in range(0, property_interaction.shape[0]):
#        if ((property_interaction['user_id'][i] == property_interaction['user_id'][y]) & (property_interaction['property_id'][i] == property_interaction['property_id'][y])):
#            sum()
    #print(property_interaction['user_id'][i])

In [ ]:
property_interaction['user_id'].nunique()

In [ ]:
property_interaction['property_id'].nunique()

In [ ]:
interactions_matrix = property_interaction.pivot(index='user_id', columns='properti_id', values='ratings')